## Get the original data

In [38]:
# The code was removed by Watson Studio for sharing.

(180, 3)

## Ignore cells with a borough that is Not assigned. 
## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [40]:
df_data_2 = df_data_1.drop(df_data_1[df_data_1.Borough == 'Not assigned'].index)
df_data_2.head(100)
df_data_2.loc[df_data_2['Neighbourhood']=='Not assigned']
df_data_2['Neighbourhood'][df_data_2['Neighbourhood']=='Not assigned']=df_data_2['Borough']
df_data_2.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## print the shape

In [41]:
df_data_2.shape

(103, 3)

## combine the latitude/longitude information 

In [42]:

body = client_4837f256871943e2ad5eaad43b31fe05.get_object(Bucket='test-donotdelete-pr-8hvz344xv5sfly',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
df_data_4 = pd.merge(left=df_data_2,right=df_data_3, on = "Postal Code")
df_data_4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_data_4['Borough'].unique()),
        df_data_4.shape[0]
    )
)

neighborhoods = df_data_4

The dataframe has 10 boroughs and 103 neighborhoods.


In [45]:
from geopy.geocoders import Nominatim 

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto City are 43.6534817, -79.3839347.


In [46]:
!pip install folium

In [47]:
import folium

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork